<a href="https://colab.research.google.com/github/khagerthey/VIS1/blob/main/Glass_vs_Plastic_Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My research mission is to investigate the glass breakages at this facility. Our largest production line is Line 3, so this current data is reflective of glass coming from that line. I believe that it is likely that the glass breakages are building up to be our most cost driven issue. From what I have seen it is only growing and I believe that it is due to thermal shock resulting from the way Chelten House handles and stores their glass.

I will be looking at the glass breakage forms from the production lines, this will give me an estimate of amount and time. I will also resource daily downtime reports from operations and the warehouse’s glass count sheets. The downtime reports will allow me to analyze how much time is lost due to glass breakages and QA process in total compared to other issues. The glass count sheets will show me how much glass we lose in storage and/or if it is being shipped to us already damaged.

--

This assignment I will be investigating the possible solution of substituting our glass jars for plastic. Plastic is a cheaper alternative, more durable, and requires less maintenance. Plastic would eliminate the glass-breakage problem entirely, or at least reduce it significantly if it were only partially transitioned to replace the most breakable glass. Although, plastic is less environmentally friendly and may upset our environmentally conscious consumers.

I will be using three sources for this assignment. My first source is a Scientific Status Summary, which provided information regarding food packaging and the qualities associated with each.
For reference: 
https://www.researchgate.net/publication/5850700_Food_PackagingRoles_Materials_and_Environmental_Issues

Second sources provided price percentage increase throughout the year:
Glass- https://data.nasdaq.com/data/FRED/PCU32723272-producer-price-index-by-industry-glass-and-glass-product-manufacturing
Plastic- https://data.nasdaq.com/data/FRED/ID3923-export-harmonized-system-plastic-articles-for-conveyence-of-packing-goods-discontinued

My third source comes from my current employer. This is an excel daily glass log that I am using to get an idea of how much glass vs. plastic jars we use for our products. I feel like this will provide insight on how big of a transition it would be to substitute glass for plastic packaging.
*Will be linking later*

In [1]:

import time, os, sys, re
import zipfile, json, datetime, string 
import numpy as np
from statistics import *

import matplotlib.pyplot as plt 
import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler

import urllib                                                 

from google.colab import files

import webbrowser                                                                      
import folium as f                                                                     
from folium.plugins import MarkerCluster, HeatMap                                      

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() 


In [38]:
glass=pd.read_csv('https://raw.githubusercontent.com/khagerthey/VIS1/main/Glass%20price%20%25%20-%20Cleaned%20(1).csv')

In [37]:
plastic=pd.read_csv('https://raw.githubusercontent.com/khagerthey/VIS1/main/Plastic%20price%20%25%20-%20Plastic%20price%20%25%20(3).csv')

I am merging together price difference percentages between glass and plastics over the years. This is to give a clear price comparison in material.

In [39]:
mergedPercent = pd.merge(glass, plastic, how='outer', on=['Date'],indicator=True)
mergedPercent

,Date,Glass Cost %,Plastic Cost %,_merge
0,2021,0.099254,-0.007517,both
1,2020,0.025830,0.012443,both
2,2019,0.015060,0.001941,both
3,2018,0.031728,0.000477,both
4,2017,0.022138,0.002763,both
5,2016,0.020996,-0.001588,both
6,2015,0.045122,-0.001748,both
7,2014,0.017227,0.000812,both
8,2013,0.013121,0.000216,both
9,2012,0.015958,0.000536,both


As we can see, glass is nearly doubling in price percentage. Glass is a more costly material and it is also more expensive to ship, as it is heavier than plastic.

Now we will be looking at the amount of weight of products generated in MSW from 1960 to 2005, with detail on containers and packaging. Weight is measured in million tons.

In [40]:
weight=pd.read_csv('https://raw.githubusercontent.com/khagerthey/VIS1/main/Copy%20of%20Food%20Packaging%20-%20Roles%2C%20Materials%2C%20and%20Environmental%20Issues%20CLEANED%20-%20Weight%20Cleaned.csv')
weight

,Date,Glass Weight,Plastic Weight
0,1960,6.2,0.1
1,1970,11.9,2.1
2,1980,14.0,3.4
3,1990,11.8,6.9
4,2000,11.0,11.9
5,2003,10.6,12.9
6,2004,10.9,14.0
7,2005,10.9,13.7


In [41]:
mergedWeight = pd.merge(weight, mergedPercent, how='outer', on=['Date'],indicator='exists')
mergedWeight

,Date,Glass Weight,Plastic Weight,Glass Cost %,Plastic Cost %,_merge,exists
0,1960,6.2,0.1,NaN,NaN,NaN,left_only
1,1970,11.9,2.1,NaN,NaN,NaN,left_only
2,1980,14.0,3.4,NaN,NaN,NaN,left_only
3,1990,11.8,6.9,NaN,NaN,NaN,left_only
4,2000,11.0,11.9,NaN,-0.000574,right_only,both
5,2003,10.6,12.9,NaN,0.003010,right_only,both
6,2004,10.9,14.0,0.005013,0.002744,both,both
7,2005,10.9,13.7,0.018766,0.001972,both,both
8,2021,NaN,NaN,0.099254,-0.007517,both,right_only
9,2020,NaN,NaN,0.025830,0.012443,both,right_only


*COME BACK AND CLEAN UP DATA, PERHAPS REMOVE MISSING/UNMATCHED VALUES!


*NEXT COME IN WITH PROPERTIES DATA
- WILL NEED TO CLEAN IT UP
- MERGE ON DATES
-- MAY NEED TO GO BACK TO PERCENTAGES AND CONDENSE INTO YEARS RATHER THAN MONTHS